In [12]:
import sys
import os
from pathlib import Path

project_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
sys.path.insert(0, str(project_root))

from core.backtesting_opt import _Data, Strategy, Backtest
from utils.functions import BlackScholes as bs
import pandas as pd
import numpy as np
import math
from scipy.optimize import root_scalar
from datetime import datetime, date as DateObject



In [13]:
class IV_Slope(Strategy):
    upper_threshold = 0.19
    lower_threshold = 0.15
    position_id= 0
    
    def init(self):
        self.legs = {
            'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg2': {'type': 'PE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg3': {'type': 'CE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg4': {'type': 'PE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None}
        }
        # Flag to track if we've already executed today
        self.executed_today = False
        self.last_execution_date = None
        self.previous_signal = 0
    
    def execution_window(self, start_time_str="14:30:00", end_time_str="15:00:00"):
        current_time = pd.Timestamp(self.time).time()
        current_date = pd.Timestamp(self.time).date()
        
        # Reset execution flag for new trading day
        if self.last_execution_date != current_date:
            self.executed_today = False
            self.last_execution_date = current_date
        
        # Check if current time is within execution window
        start_time = pd.Timestamp(start_time_str).time()
        end_time = pd.Timestamp(end_time_str).time()
        
        if not (start_time <= current_time <= end_time):
            return False
            
        # Execute only once per day during the time window
        if self.executed_today:
            return False
            
        # Mark as executed for today
        self.executed_today = True
        return True
    
    def next(self):
        if self.spot is None or pd.isna(self.spot):
            return
            
        # Check if we should execute today
        if not self.execution_window():
            return
        
        atm = round(self.spot / 50) * 50
        
        for leg in self.legs.values():
            available_ttes = list(self.tte_to_expiry.keys()) if self.tte_to_expiry else []
            lower, upper = leg["expiry_range"]
            matching_ttes = [tte for tte in available_ttes if lower <= tte <= upper]
            if not matching_ttes:
                print(f"No matching TTEs found for leg {leg['type']} with range {leg['expiry_range']}. Available TTEs: {sorted(available_ttes) if available_ttes else 'None'}")
                return 
            
            valid_tte = min(matching_ttes)
            leg["expiry"] = self.tte_to_expiry[valid_tte]
            
            leg["strike"] = float(atm)
            contract = f"NIFTY{pd.Timestamp(leg['expiry']).strftime('%d%b%y').upper()}{int(leg['strike'])}{leg['type']}"
            leg["contract"] = contract
            leg["data"] = self._data.get_ticker_data(contract)
        print(self.legs)
        missing_legs = [leg["contract"] for leg in self.legs.values() if leg["data"] is None]
        if missing_legs:
            print(f"IV not found for {self.time}. Spot: {self.spot} Missing legs: {missing_legs}")
            return
        
        iv_slope = math.log((self.legs["leg1"]["data"]["iv"] + self.legs["leg2"]["data"]["iv"]) / (self.legs["leg3"]["data"]["iv"] + self.legs["leg4"]["data"]["iv"]), 10)
        
        active_trades = self.active_trades
        new_signal = 0
        if iv_slope > self.upper_threshold:
            new_signal = -1
        elif iv_slope < self.lower_threshold:
            new_signal = 1
        
        if new_signal == -1:
            self.entry_type_dict = {'weekly': 'SELL', 'monthly': 'BUY'}
        elif new_signal == 1:
            self.entry_type_dict = {'weekly': 'BUY', 'monthly': 'SELL'}
        elif new_signal == 0:
            return
            
        
        if not active_trades:
            
            placed_any_leg = False
            for leg_id, leg in self.legs.items():
                entry_type = self.entry_type_dict.get(leg["expiry_type"])
                order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                if order_fn is None:
                    continue
                order_fn(
                    strategy_id='strat1',
                    position_id=self.position_id,
                    leg_id=leg_id,
                    ticker=leg["contract"],
                    quantity=1,
                    stop_loss=None,
                    take_profit=None,
                    tag=f'{new_signal} signal entry'
                )
                placed_any_leg = True
            if placed_any_leg:
                self.position_id += 1
                self.previous_signal = new_signal
        else:
            near_expiry = None
            for trade in active_trades:
                expiry = datetime.strptime(trade.ticker[-14:-7], "%d%b%y").date()
                near_expiry = expiry if near_expiry is None else min(near_expiry, expiry)
            
            exit_reason = (
                "Near Expiry reached" if (pd.Timestamp(self.time).date() == near_expiry) else
                "Signal changed" if (self.previous_signal != new_signal) else
                None
            )
            
            if exit_reason:
                for trade in active_trades:
                    print("Closing position")
                    trade.close(trade.size, tag=exit_reason)
                self.previous_signal = 0
                
                placed_any_leg = False
                for leg_id, leg in self.legs.items():
                    entry_type = self.entry_type_dict.get(leg["expiry_type"])
                    order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                    if order_fn is None:
                        continue
                    order_fn(
                        strategy_id='strat1',
                        position_id=self.position_id,
                        leg_id=leg_id,
                        ticker=leg["contract"],
                        quantity=1,
                        stop_loss=None,
                        take_profit=None,
                        tag=f'{new_signal} signal entry'
                    )
                    placed_any_leg = True
                if placed_any_leg:
                    self.position_id += 1
                    self.previous_signal = new_signal
                
                
            else:
                if self.previous_signal == new_signal:
                    leg_strike = self.legs["leg4"]["strike"]
                    if (0.99*leg_strike) <= self.spot <= (1.01*leg_strike):
                        pass
                    else:
                            
                            # Place orders according to entry type
                            placed_any_leg = False
                            for leg_id, leg in self.legs.items():
                                entry_type = self.entry_type_dict.get(leg["expiry_type"])
                                order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                                if order_fn is None:
                                    continue
                                print(f"Placing {entry_type} order for {leg_id}: {leg['contract']}")
                                order_fn(
                                    strategy_id='strat1',
                                    position_id=self.position_id,
                                    leg_id=leg_id,
                                    ticker=leg["contract"],
                                    quantity=1,
                                    stop_loss=None,
                                    take_profit=None,
                                    tag=f'Adjustment {entry_type} at ATM {atm}'
                                )
                                placed_any_leg = True
                            
                            if placed_any_leg:
                                self.position_id += 1

In [14]:
# db_path = "nifty_1min_desiquant.duckdb"
db_path = "nifty_opt_icici_1min.duckdb"

bt = Backtest(db_path=db_path, strategy=IV_Slope, cash=10000000, commission_per_contract=0.65, option_multiplier=75)
stats = bt.run(start_date="2022-01-01", end_date="2022-01-30")
print(stats)

Identifying tables in date range...
Found 19 tables out of 966 total tables with data in range


{'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None, 'expiry': Timestamp('2022-01-20 21:00:00'), 'strike': 17450.0, 'contract': 'NIFTY20JAN2217450CE', 'data': None}, 'leg2': {'type': 'PE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None, 'expiry': Timestamp('2022-01-20 21:00:00'), 'strike': 17450.0, 'contract': 'NIFTY20JAN2217450PE', 'data': ticker             NIFTY20JAN2217450PE
open                             270.0
high                             270.0
low                              270.0
close                            270.0
volume                          1300.0
open_interest                    250.0
strike_price                   17450.0
instrument_type                     PE
expiry_date        2022-01-20 21:00:00
Time_to_expiry                    19.0
spot_price                17357.949219
r                               0.06

{'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None, 'expiry': Timestamp('2022-01-20 21:00:00'), 'strike': 17800.0, 'contract': 'NIFTY20JAN2217800CE', 'data': ticker             NIFTY20JAN2217800CE
open                        238.350006
high                             207.0
low                              200.0
close                       233.550003
volume                         18850.0
open_interest                   4900.0
strike_price                   17800.0
instrument_type                     CE
expiry_date        2022-01-20 21:00:00
Time_to_expiry                    13.0
spot_price                     17794.5
r                               0.0653
iv                              0.1608
delta                           0.5023
gamma                         0.000739
vega                           13.3972
theta                          -8.2991
rho                             3.1076
Name: 2022-01-

{'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None, 'expiry': Timestamp('2022-01-20 21:00:00'), 'strike': 17850.0, 'contract': 'NIFTY20JAN2217850CE', 'data': ticker             NIFTY20JAN2217850CE
open                        217.300003
high                        202.949997
low                         198.800003
close                       214.050003
volume                          7400.0
open_interest                   4000.0
strike_price                   17850.0
instrument_type                     CE
expiry_date        2022-01-20 21:00:00
Time_to_expiry                    12.0
spot_price                17830.800781
r                               0.0654
iv                              0.1585
delta                           0.4911
gamma                         0.000778
vega                           12.8949
theta                          -8.5322
rho                             2.8147
Name: 2022-01-

{'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None, 'expiry': Timestamp('2022-01-27 21:00:00'), 'strike': 18200.0, 'contract': 'NIFTY27JAN2218200CE', 'data': ticker             NIFTY27JAN2218200CE
open                            258.75
high                        250.399994
low                         234.199997
close                       236.050003
volume                       1130450.0
open_interest                  63900.0
strike_price                   18200.0
instrument_type                     CE
expiry_date        2022-01-27 21:00:00
Time_to_expiry                    13.0
spot_price                18215.849609
r                               0.0656
iv                              0.1501
delta                           0.5182
gamma                         0.000772
vega                           13.7003
theta                          -7.9279
rho                              3.286
Name: 2022-01-

{'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None, 'expiry': Timestamp('2022-02-03 21:00:00'), 'strike': 18300.0, 'contract': 'NIFTY03FEB2218300CE', 'data': ticker             NIFTY03FEB2218300CE
open                        282.149994
high                             261.5
low                              261.5
close                       282.149994
volume                         16050.0
open_interest                    500.0
strike_price                   18300.0
instrument_type                     CE
expiry_date        2022-02-03 21:00:00
Time_to_expiry                    16.0
spot_price                     18278.0
r                               0.0664
iv                              0.1746
delta                           0.4945
gamma                         0.000597
vega                           15.2655
theta                          -8.3435
rho                             3.8497
Name: 2022-01-

/Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/backtesting_opt.py:736: UserWarning: Cannot determine expiry for NIFTY03FEB2217800CE. Skipping expiration check.
  warnings.warn(f"Cannot determine expiry for {ticker}. Skipping expiration check.")
/Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/backtesting_opt.py:736: UserWarning: Cannot determine expiry for NIFTY03FEB2217800PE. Skipping expiration check.
  warnings.warn(f"Cannot determine expiry for {ticker}. Skipping expiration check.")
/Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/backtesting_opt.py:736: UserWarning: Cannot determine expiry for NIFTY03FEB2217800CE. Skipping expiration check.
  warnings.warn(f"Cannot determine expiry for {ticker}. Skipping expiration check.")
/Users/vivek/Desktop/works/BACKTEST_STRATEGIES/FnO-Synapse/core/backtesting_opt.py:736: UserWarning: Cannot determine expiry for NIFTY03FEB2217800PE. Skipping expiration check.
  warnings.warn(f"Cannot determine ex

{'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None, 'expiry': Timestamp('2022-02-03 21:00:00'), 'strike': 17600.0, 'contract': 'NIFTY03FEB2217600CE', 'data': ticker             NIFTY03FEB2217600CE
open                             325.0
high                        303.049988
low                              300.0
close                       317.850006
volume                         15600.0
open_interest                   8950.0
strike_price                   17600.0
instrument_type                     CE
expiry_date        2022-02-03 21:00:00
Time_to_expiry                    12.0
spot_price                17602.300781
r                               0.0662
iv                              0.2336
delta                           0.5099
gamma                         0.000535
vega                           12.7289
theta                         -12.4075
rho                             2.8525
Name: 2022-01-

In [9]:
from core.wfo import WalkForwardOptimizer

constraint = lambda x: x['upper_threshold'] > x['lower_threshold']


wfo = WalkForwardOptimizer(
    strategy=IV_Slope,
    optimization_params={
        # 'upper_threshold': [-0.15 ,-0.1, 0.1, 0.15, 0.2, 0.25, 0.3],
        # 'lower_threshold': [-0.15 ,-0.1 ,0, 0.05, 0.1, 0.15, 0.2]
        'upper_threshold': [0.1, 0.15, 0.2, 0.25],
        'lower_threshold': [ 0.1, 0.15, 0.2]
    },
    # constraint=constraint,
    maximize='Sharpe Ratio',
)

In [ ]:
# def optimize_stock(self, stock, timeframe, db_path, exchange = None, training_candles = 2000, testing_candles = 200):
    
wfo.optimize_stock(
    stock='NIFTY',
    timeframe='1min',
    db_path='../core/nifty_1min_desiquant.duckdb',
    exchange='NSE',
    training_candles=6000,
    testing_candles=3000,
    start_date='2022-01-01',
    end_date='2022-12-31',
    # method='sambo',
    # max_tries=10
)

In [ ]:
bt.tear_sheet()